# Scenario Building
The purpose of this notebook is to understand the various methods that Network Wrangler can use to define and build scenarios.

The basic steps are:

1. Define base scenario (networks, projects)
2. Specify projects to add
3. Build the scenario

This notebook shows how this process can be used to build scenario upon scenario. 

In [1]:
import os
import sys
from network_wrangler import load_roadway, load_transit, Scenario, create_scenario

%config IPCompleter.greedy=True

import pandas as pd

pd.set_option("display.max_columns", None)

import warnings

warnings.filterwarnings("ignore")

In [2]:
%load_ext autoreload
%autoreload 2

import logging

logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# set to logging.INFO for less detail or logging.DEBUG for more
logger.setLevel(logging.INFO)

In [ ]:
# if you need to reset variables in the notebook
%reset

## Define Base Scenario

A base scenario is at a minimum a `RoadwayNetwork` and `TransitNetwork`, defined with a dictionary.

In [3]:
STPAUL_DIR = os.path.join(os.getcwd(), "../", "examples", "stpaul")
STPAUL_SHAPE_FILE = os.path.join(STPAUL_DIR, "shape.geojson")
STPAUL_LINK_FILE = os.path.join(STPAUL_DIR, "link.json")
STPAUL_NODE_FILE = os.path.join(STPAUL_DIR, "node.geojson")

road_net = load_roadway(
    links_file=STPAUL_LINK_FILE,
    nodes_file=STPAUL_NODE_FILE,
    shapes_file=STPAUL_SHAPE_FILE,
)

transit_net = load_transit(STPAUL_DIR)

2020-09-03 09:49:02, INFO: Reading from following files:
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/link.json
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/node.geojson
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/shape.geojson.
2020-09-03 09:49:02, INFO: Reading from following files:
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/link.json
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/node.geojson
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/shape.geojson.
2020-09-03 09:49:08, INFO: Read 66253 links from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/link.json
2020-09-03 09:49:08, INFO: Read 66253 

In [4]:
base_scenario = {
    "road_net": road_net,
    "transit_net": transit_net,
}

## Define "Future No Build" Scenario

In this case, we will create a future no-build scenario based on all projects in a directory with a certain filename pattern using the `glob_search` feature, but this could also be baed on a tag or whatever.

In [5]:
my_scenario_nobuild = create_scenario(
    base_scenario=base_scenario,
    card_search_dir=os.path.join(STPAUL_DIR, "project_cards"),
    glob_search="road*attribute_change*.yml",
)

2020-09-03 09:50:35, INFO: Creating Scenario
2020-09-03 09:50:35, INFO: Creating Scenario
2020-09-03 09:50:35, ERROR: Failed Project Card validation: Validation Error
2020-09-03 09:50:35, ERROR: Failed Project Card validation: Validation Error
2020-09-03 09:50:35, ERROR: Project Card File Loc:/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/project_cards/1_simple_roadway_attribute_change.yml
2020-09-03 09:50:35, ERROR: Project Card File Loc:/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/project_cards/1_simple_roadway_attribute_change.yml
2020-09-03 09:50:35, ERROR: Project Card Schema Loc:/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/network_wrangler/schemas/project_card.json
2020-09-03 09:50:35, ERROR: Project Card Schema Loc:/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/network_wrangler/schemas/project_card.json
2020-09-03 09:

**Check that the correct projects got added**

- These projects haven't been applied (or "built") yet, they are just specified at this point.

In [6]:
my_scenario_nobuild.queued_projects

['Improve Express Bus Frequency',
 '6th St E Road Diet',
 'Blue Line Service Enhancement D',
 'Blue Line Service Enhancement B',
 'Blue Line Service Enhancement C',
 'Blue Line Service Enhancement A',
 'Improve Express Bus Frequency B',
 '6th Street Transitway']

## Build the "Future No Build" Scenario
 - This is where you are actually "building" the projects
 - You can check to see *all* the built projects using the `applied_projects` attribute

In [8]:
my_scenario_nobuild.apply_all_projects()

2020-09-03 09:53:17, INFO: Applying 6th Street Transitway
2020-09-03 09:53:17, INFO: Applying 6th Street Transitway
2020-09-03 09:53:17, INFO: Applying Project to Roadway Network: 6th Street Transitway
2020-09-03 09:53:17, INFO: Applying Project to Roadway Network: 6th Street Transitway
2020-09-03 09:53:17, WARNING: Existing value defined for lanes in project card does not match the value in the roadway network for the selected links
2020-09-03 09:53:17, WARNING: Existing value defined for lanes in project card does not match the value in the roadway network for the selected links
2020-09-03 09:53:17, INFO: Applying Improve Express Bus Frequency B
2020-09-03 09:53:17, INFO: Applying Improve Express Bus Frequency B
2020-09-03 09:53:17, INFO: Applying Project to Transit Network: Improve Express Bus Frequency B
2020-09-03 09:53:17, INFO: Applying Project to Transit Network: Improve Express Bus Frequency B
2020-09-03 09:53:17, INFO: Applying Blue Line Service Enhancement A
2020-09-03 09:53

In [9]:
my_scenario_nobuild.applied_projects

['6th Street Transitway',
 'Improve Express Bus Frequency B',
 'Blue Line Service Enhancement A',
 'Blue Line Service Enhancement C',
 'Blue Line Service Enhancement B',
 'Blue Line Service Enhancement D',
 '6th St E Road Diet',
 'Improve Express Bus Frequency']

## Define "Future Build" Scenario

 - Specify specific cards to add for a build scenario
 - This scenario will be tiered off of the no-build scenario and will have 'knowledge' of what projects have already been applied when checking pre-requisites.
 

 - Here, we are going to specify a list of very specific build projects.
 - In this case, we need to create `ProjectCard` instances from the YML files and then add them to the `project_card_list` keyword. 

In [10]:
BUILD_CARD_FILENAMES = [
    "road.managed_lane.simple.yml",
]

In [11]:
my_scenario_build_alt1 = Scenario.create_scenario(
    base_scenario=my_scenario_nobuild.__dict__,
    project_card_filepath=BUILD_CARD_FILENAMES,
)

my_scenario_build_alt1.applied_projects

2020-09-03 09:56:47, INFO: Creating Scenario
2020-09-03 09:56:47, INFO: Creating Scenario


['6th Street Transitway',
 'Improve Express Bus Frequency B',
 'Blue Line Service Enhancement A',
 'Blue Line Service Enhancement C',
 'Blue Line Service Enhancement B',
 'Blue Line Service Enhancement D',
 '6th St E Road Diet',
 'Improve Express Bus Frequency']

In [12]:
## Apply Build Scenario Projects

In [13]:
my_scenario_build_alt1.apply_all_projects()

2020-09-03 09:57:07, INFO: Applying test managed lane project
2020-09-03 09:57:07, INFO: Applying test managed lane project
2020-09-03 09:57:07, INFO: Applying Project to Roadway Network: test managed lane project
2020-09-03 09:57:07, INFO: Applying Project to Roadway Network: test managed lane project


In [14]:
my_scenario_build_alt1.applied_projects

['6th Street Transitway',
 'Improve Express Bus Frequency B',
 'Blue Line Service Enhancement A',
 'Blue Line Service Enhancement C',
 'Blue Line Service Enhancement B',
 'Blue Line Service Enhancement D',
 '6th St E Road Diet',
 'Improve Express Bus Frequency',
 'test managed lane project']